In [ ]:
import pandas as pd
import nltk
import re
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [ ]:
# /teamspace/studios/this_studio/sentiment_analysis-project
train_data = './sentiment_analysis-project/data/train.csv'
test_data = './sentiment_analysis-project/data/test.csv'
valid_data = './sentiment_analysis-project/data/validation.csv'

file = './sentiment_analysis-project/multiclass_dataset.csv'

def save_load_df(file:str):
    if os.path.exists(file):
        df = pd.read_csv(file, index_col= 0)
    else:
        df = pd.concat(map(pd.read_csv, [train_data, test_data, valid_data]), axis= 0, ignore_index=True)
        df.to_csv(file, columns= ['id', 'text', 'label', 'sentiment'])
        df = pd.read_csv(file, index_col= 0)
    return df

df = save_load_df(file=file)
df

## TP1, TP2

In [ ]:
class PreProcess():
    def __init__(self):
        self.text_pattern = re.compile(
        r'(<.+?>)'         # Balises HTML
        r'(#|@)\w+'  # @ and # words
        r'|([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})'  # Emails
        r'|(https?://[^\s\n\r]+)' # URLs commençant par http ou https
        r'|(www\.[^\s]+)'      # URLs commençant par www
        r'|([\U00010000-\U0010ffff])'  # Émojis et autres caractères au-delà de l'ASCII étendu
        r'|([^\x00-\xFF])'     # Tout ce qui n'est pas en ASCII étendu (0-255)
        )
        self.emoji_pattern = re.compile(
            "[\U0001F600-\U0001F64F"  # Emoticons
            "\U0001F300-\U0001F5FF"  # Miscellaneous Symbols and Pictographs
            "\U0001F680-\U0001F6FF"  # Transport and Map Symbols
            "\U0001F700-\U0001F77F"  # Alchemical Symbols
            "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
            "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
            "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
            "\U0001FA00-\U0001FA6F"  # Chess Symbols
            "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
            "\U00002702-\U000027B0"  # Dingbats
            "\U000024C2-\U0001F251"  # Enclosed characters
            "]+",
            flags=re.UNICODE,
        )
        self.punctuation = set(string.punctuation)
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()

    def clean_text(self, text):
        text = self.text_pattern.sub('', str(text))
        text = self.emoji_pattern.sub('', str(text))
        text = text.lower()
        return text
    
    def get_tokens(self, text):
        text = self.clean_text(text)
        sentences = nltk.sent_tokenize(text)
        tokens = []
        for sentence in sentences:
            words = nltk.word_tokenize(sentence)
            for word in words:
                if word not in self.stop_words:
                    # clean all the punctuation and the StopWords
                    word = ''.join([c for c in word if c not in self.punctuation])
                    if word == '':
                        continue
                    tokens.append(word)
        return tokens

    def lemmetize_with_pos(self, tokens):
        pos_tags = nltk.pos_tag(tokens)
        lemmes = [] 
        pos_tag = []
        for token, pos in pos_tags:
            if pos.startswith('J'):
                lemma = self.lemmatizer.lemmatize(token, pos = 'a')
            elif pos.startswith('V'):
                lemma =  self.lemmatizer.lemmatize(token, pos = 'v')
            elif pos.startswith('RB'):
                lemma = self.lemmatizer.lemmatize(token, pos = 'r')
            elif pos.startswith('N'):
                lemma = self.lemmatizer.lemmatize(token, pos = 'n')
            else:
                lemma = self.lemmatizer.lemmatize(token)
            lemmes.append(lemma)
            pos_tag.append(pos)
        return lemmes, pos_tag
    
    def get_lemmes(self, text):
        tokens = self.get_tokens(text)
        lemmes, _ = self.lemmetize_with_pos(tokens)
        return lemmes
    
    def visualize_data(self, text):
        text = self.clean_text(text)
        tokens = self.get_tokens(text)
        lemmes, pos_tag = self.lemmetize_with_pos(tokens)
        data = [[token, lemme, pos] for token, lemme, pos in zip(tokens, lemmes, pos_tag)]
        return data
    
texts = list(df['text'])
labels = list(df['label'])
process_text = PreProcess()
corpus = pd.DataFrame(data = [(process_text.get_tokens(text), label) for text, label in zip(texts, labels)], columns=['sentence', 'label'])
corpus

In [ ]:
# drop the empty rows
corpus = corpus[corpus['sentence'].str.len() != 0]
corpus = corpus.reset_index(drop = True)
corpus

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_name = 'distilbert-base-uncased'
# model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name)
bert_model.to(device)

In [ ]:
sentences = list(corpus['sentence'])
sentences = [' '.join(sentence) for sentence in sentences]
X = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

labels = list(corpus['label'])
Y = torch.tensor(labels, dtype = torch.long)

### option 1

In [ ]:
# x = {k: v.to(device) for k, v in X.items()}
# with torch.no_grad():
#     x = bert_model(**x)
#     x = x.last_hidden_state
        

### Option 2

In [ ]:
X['labels'] = Y
train_data, valid_data, test_data = {}, {}, {}
for k, v in X.items():
    train_data[k], test_data[k] = train_test_split(v, test_size=0.3, train_size=0.7, random_state=42, shuffle=True)
    test_data[k], valid_data[k] = train_test_split(test_data[k], test_size=0.5, train_size=0.5, random_state=42, shuffle=True)

class DictDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        return {
            'input_ids': self.data['input_ids'][idx],
            'attention_mask': self.data['attention_mask'][idx],
            # 'token_type_ids': self.data['token_type_ids'][idx]
              }, self.data['labels'][idx]

train_data = DictDataset(train_data)
valid_data = DictDataset(valid_data)
test_data = DictDataset(test_data)



In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

batch_size = 256
train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
valid_dl = DataLoader(valid_data, batch_size=batch_size, shuffle=False, drop_last=True)
test_dl = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=False)


class LSTModel(nn.Module):
    def __init__(self,
                 embed_dim = 768,
                 hidden_size = 128,
                 bidirectional = False,
                 num_layers = 1,
                 dropout = 0.4
        ):
        super(LSTModel, self).__init__()
        self.bidirectional = bidirectional
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size= embed_dim,
                            hidden_size = hidden_size,
                            num_layers=num_layers,
                            bias= False,
                            bidirectional=bidirectional,
                            batch_first=True,
                            dropout=dropout if num_layers > 1 else 0
        )
        self.D = 1 if self.bidirectional == False else 2
        self.fc2 = nn.Linear(self.D * hidden_size, 3)

    def forward(self, x):
        # B, T, C = x.shape
        # h0 = torch.zeros((self.D, self.num_layers, B, self.hidden_size), device=device).view(-1, B, self.hidden_size)
        # c0 = torch.zeros((self.D, self.num_layers, B, self.hidden_size), device=device).view(-1, B, self.hidden_size)
        out, (h1, c1) = self.lstm(x)#, (h0, c0))
        out = F.softmax(self.fc2(out), dim = -1)
        return out

torch.manual_seed(42) # for the distribution of the gradient

torch.set_float32_matmul_precision('high')

model = LSTModel()
model.to(device=device)

optimizer = optim.Adam(model.parameters(), lr = 3e-4)


In [ ]:
import time

train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []
num_iter = 50
for i in range(num_iter):
    t1 = time.time()
    train_loss = 0
    train_acc = 0
    model.train()
    for xb_train, yb_train in train_dl:
        x = {k: v.to(device) for k, v in xb_train.items()}
        y = yb_train.to(device)
        
        with torch.no_grad():
            with torch.autocast(device_type = 'cuda', dtype = torch.bfloat16):
                x = bert_model(**x)
                x = x.last_hidden_state
        
        optimizer.zero_grad()
        with torch.autocast(device_type = 'cuda', dtype = torch.bfloat16):
            logits = model(x)[:, -1, :]
            loss = F.cross_entropy(logits, y)
        loss.backward()
        train_loss += loss.item() * batch_size
        pred = torch.argmax(logits, dim = 1)
        train_acc += (pred == y).float().sum().item()
        optimizer.step()
    train_loss /= len(train_dl.dataset)
    train_acc /= len(train_dl.dataset)
    model.eval()
    with torch.no_grad():
        valid_loss = 0
        valid_acc = 0
        for xb_valid, yb_valid in valid_dl:
            xb_valid = {k: v.to(device) for k, v in xb_valid.items()}
            yb_valid = yb_valid.to(device)
            xb_valid = bert_model(**xb_valid)
            xb_valid = xb_valid.last_hidden_state
            logits = model(xb_valid)[:, -1, :]
            loss = F.cross_entropy(logits, yb_valid)
            valid_loss += loss.item() * batch_size
            pred = torch.argmax(logits, dim = 1)
            valid_acc += (pred == yb_valid).float().sum().item()
    valid_loss /= len(valid_dl.dataset)
    valid_acc /= len(valid_dl.dataset)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    train_accuracies.append(train_acc)
    valid_accuracies.append(valid_acc)
    t2 = time.time()
    dt = (t2 - t1)
    print(f'epoch {i} || train_loss {train_loss:4f}, valid_loss {valid_loss:4f}, train_accu {train_acc:4f} , valid_accu {valid_acc:4f} , time {dt:4f}')

In [ ]:
import matplotlib.pyplot as plt

x_arr = np.arange(num_iter) + 1

fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, train_losses, '-o', label='Train loss')
ax.plot(x_arr, valid_losses, '--<', label='Validation loss')
ax.legend(fontsize=10)
ax.set_xlabel('Epoch', size=10)
ax.set_ylabel('Loss', size=10)

ax = fig.add_subplot(1, 2, 2)
ax.plot(x_arr, train_accuracies, '-o', label='Train acc.')
ax.plot(x_arr, valid_accuracies, '--<', label='Validation acc.')
ax.legend(fontsize=10)
ax.set_xlabel('Epoch', size=10)
ax.set_ylabel('Accuracy', size=10)

plt.show()

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

test_acc = 0
all_labels = []
all_preds = []
with torch.no_grad():
    for xb_test, yb_test in test_dl:
        xb_test = xb_test.to(device)
        yb_test = yb_test.to(device)
        logits = model(xb_test)[:, -1, :]
        pred = torch.argmax(logits, dim = 1)
        test_acc += (pred == yb_test).float().sum().item()
        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(yb_test.cpu().numpy())

test_acc /= len(test_dl.dataset)
precision = precision_score(all_labels, all_preds, average='weighted')  # 'weighted' pour pondérer selon les classes
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print('test accuracy', test_acc)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("\nClassification Report:\n", classification_report(all_labels, all_preds))